In [ ]:

import numpy as np
import h5py
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

#IMPORT ALL PRE BUILD FUNCTION 
from func import *
from momentum import *
from RMSprop import *
from adam import *
from matplotlib import pyplot as plt

In [6]:
hf1 = h5py.File('/pythonfile/popular_optimizer/train_datasets.h5','r')  #READ THE DATASETS
X_train = hf1.get('X_train')     #GET THE X TRAINING DATASETS
Y_train = hf1.get('Y_train')
#flatten the array and normalize 
X_train = np.reshape(X_train,(X_train.shape[0],-1))  #FLATTEN   (209, 64 *64 * 3)
X_train = preprocessing.normalize(X_train)           #NORMALIZING THE DATASETS

Y_train = np.reshape(Y_train,(Y_train.shape[0],1))                # MAKING SURE IT IS IN CORRECT SHAPE

print("Shape of X_train {}  and  Y_train {}".format(X_train.shape,Y_train.shape))

Shape of X_train (259, 12288)  and  Y_train (259, 1)


In [8]:
def model(X,Y,learning_rate,num_iter,hidden_size,keep_prob,optimizer):
    L = len(hidden_size)
    params = initilization(X.shape[1], hidden_size)
    costs = []
    itr  = []

    if optimizer == 'momentum':
        v = initilization_moment(params)

    elif optimizer == 'rmsprop':
        s = initilization_RMS(params)

    elif optimizer == 'adam' :
        v,s = initilization_Adam(params)

    for i in range(1,num_iter):
        MiniBatches = RandomMiniBatches(X, Y, 32)   # GET RAMDOMLY MINIBATCHES
        p , q = MiniBatches[2]
        for MiniBatch in MiniBatches:               #LOOP FOR MINIBATCHES

            (MiniBatch_X, MiniBatch_Y) = MiniBatch

            cache, A = model_forward(MiniBatch_X, params, L,keep_prob)     #FORWARD PROPOGATIONS
            cost = cost_f(A, MiniBatch_Y)                                  #COST FUNCTION
            grad = backward(MiniBatch_X, MiniBatch_Y, params, cache, L,keep_prob) #BACKWARD PROPAGATION 

            if optimizer == 'momentum':
                params = update_params_with_momentum(params, grad, v, beta=0.9,learning_rate=learning_rate)

            elif optimizer == 'rmsprop':
               params = update_params_with_RMS(params, grad, s, beta=0.9,learning_rate=learning_rate)

            elif optimizer == 'adam' :
                params = update_params_with_Adam(params, grad,v, s, beta1=0.9,beta2=0.999,  learning_rate=learning_rate,t=i)                                         #UPDATE PARAMETERS
            elif optimizer == "minibatch":
                params = update_params(params, grad,learning_rate=learning_rate) 

           
        
        if i%10 == 0:
            costs.append(cost)
            itr.append(i)
            if i % 100 == 0 :
                print('cost of iteration______{}______{}'.format(i,cost))
    return params,costs,itr

In [9]:
params, cost_sgd,itr = model(X_train, Y_train, learning_rate = 0.01,
               num_iter=1000, hidden_size=[100, 1],keep_prob=1,optimizer='minibatch')
Y_train_pre = predict(X_train, params, 2)
print('train_accuracy------------', accuracy_score(Y_train_pre, Y_train))

cost of iteration______100______0.35302967575683797


KeyboardInterrupt: 

In [ ]:
params,cost_momentum, itr = model(X_train, Y_train, learning_rate = 0.01,
               num_iter=1000, hidden_size=[100, 1],keep_prob=1,optimizer='momentum')
Y_train_pre = predict(X_train, params, 2)
print('train_accuracy------------', accuracy_score(Y_train_pre, Y_train))

In [ ]:
params,cost_rms,itr = model(X_train, Y_train, learning_rate = 0.01,
               num_iter=1000, hidden_size=[100, 1],keep_prob=1,optimizer='rmsprop')
Y_train_pre = predict(X_train, params, 2)
print('train_accuracy------------', accuracy_score(Y_train_pre, Y_train))

In [ ]:
params,cost_adam, itr = model(X_train, Y_train, learning_rate = 0.01,
               num_iter=1000, hidden_size=[100, 1],keep_prob=1,optimizer='adam')
Y_train_pre = predict(X_train, params, 2)
print('train_accuracy------------', accuracy_score(Y_train_pre, Y_train))

In [ ]:
plot_fn(itr,cost_momentum,cost_rms,cost_adam,cost_sgd)